In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms 

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

In [4]:
#grabbing the mnist image files and converting them into tensors 

In [5]:
transform=transforms.ToTensor()

In [8]:
train_data=datasets.MNIST(root='C:\\Users\\DELL\\OneDrive\\Desktop', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:11<00:00, 826432.02it/s] 


Extracting C:\Users\DELL\OneDrive\Desktop\MNIST\raw\train-images-idx3-ubyte.gz to C:\Users\DELL\OneDrive\Desktop\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 113529.98it/s]


Extracting C:\Users\DELL\OneDrive\Desktop\MNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\DELL\OneDrive\Desktop\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:04<00:00, 393257.98it/s]


Extracting C:\Users\DELL\OneDrive\Desktop\MNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\DELL\OneDrive\Desktop\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3008611.62it/s]

Extracting C:\Users\DELL\OneDrive\Desktop\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\DELL\OneDrive\Desktop\MNIST\raw



In [9]:
test_data=datasets.MNIST(root='C:\\Users\\DELL\\OneDrive\\Desktop', train=False, download=True, transform=transform)

In [10]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: C:\Users\DELL\OneDrive\Desktop
    Split: Train
    StandardTransform
Transform: ToTensor()

In [11]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: C:\Users\DELL\OneDrive\Desktop
    Split: Test
    StandardTransform
Transform: ToTensor()

In [12]:
type(train_data)

torchvision.datasets.mnist.MNIST

In [14]:
type(train_data[0])

tuple

In [15]:
image, label=train_data[0]

In [17]:
image.shape

torch.Size([1, 28, 28])

In [19]:
label

5

In [20]:
torch.manual_seed(101)
train_loader=DataLoader(train_data, batch_size=100, shuffle=True)
test_loader=DataLoader(test_data, batch_size=500, shuffle=False)

#batch_size is the nmber of records we process at a time. we set shuffle to true in training because sometimes, datasets are sorted,
#the model might get loads of 0's first, followed by loads of 1's which is not what we want. 
#in the testing, we don't have to adjust weights, so we set the batch_size to a little higher
#we also don't care what order the images come in, so there is no need to shuffle 


In [21]:
from torchvision.utils import make_grid
np.set_printoptions(formatter=dict(int=lambda x: f'{x:4}')) #formatting 

In [22]:
#first batch 
for images, labels in train_loader:
    #60,000/100 == 60 times loop runs 
    break #because we only want the first batch 
    

In [23]:
images.shape

torch.Size([100, 1, 28, 28])

In [25]:
labels.shape

torch.Size([100])

In [28]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, in_sz=784, out_sz=10, layers=[120, 84]):
        super().__init__()
        self.fc1=nn.Linear(in_sz, layers[0])
        self.fc2=nn.Linear(layers[0], layers[1])
        self.fc3=nn.Linear(layers[1], out_sz)
        
        
    def forward(self, X):
        X=F.relu(self.fc1(X))
        X=F.relu(self.fc2(X))
        X=self.fc3(X)
        
        return F.log_softmax(X, dim=1) #becuase it is a multi class classiication 
    

In [29]:
torch.manual_seed(101)
model=MultiLayerPerceptron()
model

MultiLayerPerceptron(
  (fc1): Linear(in_features=784, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [30]:
 for param in model.parameters():
        print(param.numel()) #making it print the number of parameters 
#in ann, if you sum up all those below, we have a huge number 

94080
120
10080
84
840
10


In [31]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
images.shape

torch.Size([100, 1, 28, 28])

In [33]:
#need to combine that 1,28,28 into 784. 
images.view(100,-1).shape

torch.Size([100, 784])

In [35]:
import time
start_time=time.time()

#training 
epochs=10
train_losses=[]
test_losses=[]
train_correct=[]
test_correct=[]

for i in range (epochs):
    trn_corr=0
    tst_corr=0
    for b, (X_train, y_train) in enumerate(train_loader): #b gives batch number, X_train gives the image, y_train gives the answer
        b+=1
        
        y_pred=model(X_train.view(100,-1))
        loss=criterion(y_pred, y_train)
        predicted=torch.max(y_pred.data, 1)[1] #the output layer will actually give out probabilities for each number. we pick the one with the highest probability and the [1] will give the number 
        batch_corr=(predicted==y_train).sum()
        trn_corr+=batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%200==0:
            acc=trn_corr.item()*100/(100*b)
            print(f'Epoch {i} batch {b} loss:{loss.item()} accuracy:{acc}')
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    with torch.no_grad(): #no_grad here, not zero_grad because we don't want to update the weights in testing 
        for b,(X_test, y_test) in enumerate(test_loader):
            y_val=model(X_test.view(500,-1))
            predicted=torch.max(y_val.data,1)[1]
            tst_corr+=(predicted==y_test).sum()
            
    loss=criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
            
        


total_time=time.time()-start_time
print(f'Duration: {total_time/60} mins')

Epoch 0 batch 200 loss:0.24509955942630768 accuracy:94.88
Epoch 0 batch 400 loss:0.14002777636051178 accuracy:95.1075
Epoch 0 batch 600 loss:0.07295873761177063 accuracy:95.38333333333334
Epoch 1 batch 200 loss:0.08890343457460403 accuracy:96.575
Epoch 1 batch 400 loss:0.06103847920894623 accuracy:96.6475
Epoch 1 batch 600 loss:0.06553444266319275 accuracy:96.72666666666667
Epoch 2 batch 200 loss:0.11172612011432648 accuracy:97.665
Epoch 2 batch 400 loss:0.051425594836473465 accuracy:97.53
Epoch 2 batch 600 loss:0.03153930604457855 accuracy:97.55
Epoch 3 batch 200 loss:0.11145282536745071 accuracy:98.145
Epoch 3 batch 400 loss:0.0499701052904129 accuracy:97.9775
Epoch 3 batch 600 loss:0.05861545726656914 accuracy:97.98666666666666
Epoch 4 batch 200 loss:0.007547399960458279 accuracy:98.35
Epoch 4 batch 400 loss:0.040722399950027466 accuracy:98.4825
Epoch 4 batch 600 loss:0.0627918541431427 accuracy:98.41166666666666
Epoch 5 batch 200 loss:0.02982754074037075 accuracy:98.845
Epoch 5 bat

In [36]:
test_load_all=DataLoader(test_data, batch_size=10000, shuffle=False)


In [38]:
with torch.no_grad():
    correct=0
    for X_test, y_test in test_load_all:
        y_val=model(X_test.view(len(X_test), -1))
        predicted=torch.max(y_val, 1)[1]
        correct+=(predicted==y_test).sum()

In [41]:
100*correct.item()/len(test_data)

97.74

In [42]:
confusion_matrix(predicted.view(-1), y_test.view(-1))

array([[ 973,    0,    3,    0,    2,    2,    8,    1,    3,    2],
       [   0, 1130,    1,    0,    0,    0,    3,    5,    1,    2],
       [   0,    3, 1004,    3,    2,    0,    3,    6,    4,    0],
       [   2,    1,    3,  993,    0,   15,    1,    1,   15,    3],
       [   1,    0,    2,    0,  958,    2,    3,    1,    3,   13],
       [   0,    0,    0,    3,    1,  866,   10,    0,    5,    2],
       [   1,    0,    2,    0,    4,    0,  930,    0,    1,    0],
       [   1,    0,   11,    6,    4,    1,    0, 1009,    5,    7],
       [   2,    1,    6,    2,    1,    5,    0,    1,  935,    4],
       [   0,    0,    0,    3,   10,    1,    0,    4,    2,  976]],
      dtype=int64)